In [30]:
import numpy as np
import os
import pandas as pd
import arcpy

In [31]:
working_directory = os.getcwd()
print(working_directory)

E:\GitHub\TAZ-to-ZIPCode


In [33]:
sde_filename = "agrc_opensgid"

#make only once
arcpy.CreateDatabaseConnection_management (working_directory, sde_filename, "POSTGRESQL", "opensgid.agrc.utah.gov", "DATABASE_AUTH", "agrc", "agrc", "SAVE_USERNAME", "opensgid")

In [34]:
zipcodearea_layername = "opensgid.boundaries.zip_code_areas"

zipcode = sde_filename + ".sde\\" + zipcodearea_layername

print(arcpy.Describe(zipcode).shapeType)



Polygon


<Result 'opensgid.boundaries.zip_code_areas'>

Copy locally to make further processing faster

In [42]:
out_featureclass = os.path.join(working_directory + "\\ZipCode\\", "ZipCode")
arcpy.CopyFeatures_management(zipcode, out_featureclass)

<Result 'E:\\GitHub\\TAZ-to-ZIPCode\\ZipCode\\ZipCode.shp'>

In [47]:
arcpy.MakeFeatureLayer_management(out_featureclass + ".shp","ZC")

<Result 'ZC'>

In [45]:
arcpy.MakeFeatureLayer_management(working_directory + "\\TAZ\\TAZ.shp", "TAZ")

<Result 'TAZ'>